<table>
    <tr>
        <td style="text-align:left">
            <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR9ItLTT_F-3Q30cu7ZCCoKmuFGBt22pe7pNA" alt="Logo Universidad" width="300"/>
        </td>
        <td>
            Departamento de Ciencias de la Computación y de la Decisión<br>
            Facultad de Minas<br>
            Universidad Nacional de Colombia<br>
            Optimizacion e IA 2024-2S<br><br>
            Docente: Maria Constanza Torres Madronero<br>
            <br>
            Contribuciones a la guia por: <br>
            - Deimer Miranda Montoya (2023)<br>
            - Luis Fernando Becerra Monsalve (2024)
        </td>    
    </tr>
</table>

# **Clasificacion supervisada**

In [28]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import pandas as pd


In [29]:
#Cargar los datos
iris = datasets.load_iris()

#Organizamos los datos en un DataFrame de panda
data = pd.DataFrame({
    'sepal length' :iris.data[:,0],
    'sepal width' :iris.data[:,1],
    'petal length' :iris.data[:,2],
    'petal width' :iris.data[:,3],
    'species' :iris.target
})

In [30]:
#Dividimos los datos entre las muestras de entrenamiento y prueba
X = data[['sepal length', 'sepal width', 'petal length', 'petal width']] #Muestras
#X = data[['sepal length', 'petal length', 'petal width']] #Muestras
y = data['species'] #Etiquetas
#Seleccionamos 70% de muestras para entrenamiento y 30% para prueba
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)


#### (c) Optimizacion de hiperparametros usando busqueda de grilla

In [ ]:
#Vamos a seleccionar los hiperparametros de nuestro Random Forest empleando la busqueda
#de grilla. La libreria sklearn incluye este metodo


#Creamos la grilla con los parametros a seleccionar


#Defimos el clasificador base

#Realizamos la busqueda de los hiperparametros

#Nota: no se olviden de invocar el metodo para entrenar .fit


In [ ]:
#Podemos ver que parametros fueron los seleccionados



In [ ]:
#Usamos el clasificador y analizamos su precision


In [ ]:
#Otra alternativa para analizar el desempeño es la matriz de confusion
#tambien podemos calcularla desde la libreria sklearn



(d) Optimizacion de hiperparametros usando busqueda aleatoria

In [ ]:
#Ahora, vamos a seleccionar los hiperparametros empleando busqueda aleatoria
#este tipo de optimizador tambien se encuentra en sklearn

#Creamos la grilla de hiperparametros, entre las cuales se seleccionara de forma
#aleatoria la mejor configuracion




#Definimos el clasificador base

#Seleccionamos los hiperparametros

#Nota: despues de emplear RandomizedSearchVC tambien se requiere entrenar
#el modelo


In [ ]:
#Revisamos los parametros seleccionados


In [ ]:
#Calculamos la precision
y_test=RF_randomSearch.predict(X_test)
print("Accuracy - Opt", metrics.accuracy_score(y_test,y_pred))

In [ ]:
#Calculamos la matriz de confusion
cm_rf = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_rf, display_labels=RF_gridSearch.best_estimator_.classes_)
disp.plot()

#### Anotaciones importantes
1.	Nota que cada vez que corres el programa, cambian las muestras de entrenamiento y prueba, ya que estas son seleccionadas aleatoriamente. Por tanto, cambian los resultados de precisión. Para evitar el sesgo en el entrenamiento se debe incorporar métodos de validación cruzada.
2.	Compara los tiempos de ejecución tanto de la búsqueda en grilla como la búsqueda aleatoria. Usualmente, la búsqueda aleatoria converge mas rápido que la búsqueda por grilla.
3.	Si decides cambiar el espacio de representación, es decir, trabajar con un subconjunto de las características, es necesario volver a optimizar los hiperparametros. En clasificadores con SVM, RG e incluso NN pequeñas este proceso puede no ser costoso, pero en arquitecturas complejas puede tomar mucho tiempo.
